# TCGA Kidney scRNA-seq Create h5ad file

This notebook reads in scRNA-seq data from a GDC `.tar.gz` download and prepares it for downstream analysis.

In [ ]:
import os
import tarfile
import glob
import scanpy as sc
import anndata as ad
import pandas as pd
import loompy
import numpy as np
import h5py

sc.settings.verbosity = 3
sc.logging.print_header()

## 1. Define Paths

In [ ]:
# Path to the .tar.gz file
# Set KIDNEY_TARBALL_PATH env var to override, e.g.:
#   export KIDNEY_TARBALL_PATH="/path/to/your/tarball.tar.gz"
# Path to the .tar.gz file on Google Drive
TARBALL_PATH = os.environ.get(
    "KIDNEY_TARBALL_PATH"
)

# Local directory to extract contents into
EXTRACT_DIR = os.path.join(os.getcwd(), "data", "gdc_extract")
os.makedirs(EXTRACT_DIR, exist_ok=True)

print(f"Tarball path: {TARBALL_PATH}")
print(f"Extract dir:  {EXTRACT_DIR}")
print(f"Tarball exists: {os.path.exists(TARBALL_PATH)}")
print(f"Tarball size:  {os.path.getsize(TARBALL_PATH) / 1e9:.2f} GB")

## 2. Inspect the Tarball Contents

Before extracting, let's list the files inside the archive to understand its structure.

In [ ]:
# List the contents of the tarball without extracting
with tarfile.open(TARBALL_PATH, "r:gz") as tar:
    members = tar.getmembers()
    print(f"Total files in archive: {len(members)}\n")
    for m in members:
        size_mb = m.size / 1e6
        print(f"  {m.name}  ({size_mb:.2f} MB)")

## 3. Extract the Tarball

In [ ]:
# Extract all files to the local extract directory
with tarfile.open(TARBALL_PATH, "r:gz") as tar:
    tar.extractall(path=EXTRACT_DIR)

print("Extraction complete!")
print(f"\nExtracted contents:")
for root, dirs, files in os.walk(EXTRACT_DIR):
    level = root.replace(EXTRACT_DIR, "").count(os.sep)
    indent = "  " * level
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = "  " * (level + 1)
    for f in files:
        fpath = os.path.join(root, f)
        size_mb = os.path.getsize(fpath) / 1e6
        print(f"{sub_indent}{f}  ({size_mb:.2f} MB)")

## 4. Load the Data

GDC scRNA-seq downloads typically contain `.h5` (10x HDF5), `.h5ad`, or `.loom` files. The cell below auto-detects the file type and loads accordingly.

In [ ]:
# Discover all relevant data files
h5_files = glob.glob(os.path.join(EXTRACT_DIR, "**", "*.h5"), recursive=True)
h5ad_files = glob.glob(os.path.join(EXTRACT_DIR, "**", "*.h5ad"), recursive=True)
loom_files = glob.glob(os.path.join(EXTRACT_DIR, "**", "*.loom"), recursive=True)
mtx_dirs = glob.glob(os.path.join(EXTRACT_DIR, "**", "matrix.mtx*"), recursive=True)
tsv_files = glob.glob(os.path.join(EXTRACT_DIR, "**", "*.tsv*"), recursive=True)

print(f"Found {len(h5_files)} .h5 files")
print(f"Found {len(h5ad_files)} .h5ad files")
print(f"Found {len(loom_files)} .loom files")
print(f"Found {len(mtx_dirs)} matrix.mtx files")
print(f"Found {len(tsv_files)} .tsv files")

for f in h5_files:
    print(f"  .h5:   {f}")
for f in h5ad_files:
    print(f"  .h5ad: {f}")
for f in loom_files:
    print(f"  .loom: {f}")

In [ ]:
# Load the data based on detected file type
adata_list = []

if h5ad_files:
    # Load .h5ad files directly
    for f in h5ad_files:
        print(f"Loading h5ad: {os.path.basename(f)}")
        adata_list.append(sc.read_h5ad(f))

elif h5_files:
    # Load 10x HDF5 files
    for f in h5_files:
        print(f"Loading 10x h5: {os.path.basename(f)}")
        try:
            adata_list.append(sc.read_10x_h5(f))
        except Exception as e:
            print(f"  Could not read as 10x h5, trying generic h5...")
            # Inspect the HDF5 structure for non-standard formats
            with h5py.File(f, "r") as h5f:
                print(f"  HDF5 keys: {list(h5f.keys())}")
                def print_h5_structure(name, obj):
                    print(f"    {name}: {type(obj).__name__}")
                h5f.visititems(print_h5_structure)

elif loom_files:
    # Load .loom files
    for f in loom_files:
        print(f"Loading loom: {os.path.basename(f)}")
        adata_list.append(sc.read_loom(f))

elif mtx_dirs:
    # Load from Market Matrix format (10x style)
    for mtx_file in mtx_dirs:
        mtx_parent = os.path.dirname(mtx_file)
        print(f"Loading 10x mtx from: {mtx_parent}")
        adata_list.append(sc.read_10x_mtx(mtx_parent))

else:
    print("No recognized data files found. Please inspect the extracted directory above.")

print(f"\nLoaded {len(adata_list)} AnnData object(s)")

In [ ]:
# If multiple AnnData objects, concatenate; otherwise use the single one
if len(adata_list) == 1:
    adata = adata_list[0]
elif len(adata_list) > 1:
    print("Concatenating multiple AnnData objects...")
    adata = ad.concat(adata_list, join="outer", label="batch")
    adata.obs_names_make_unique()
else:
    raise ValueError("No data was loaded. Check the extracted files above.")

print(f"\nAnnData object:")
print(adata)
print(f"\nShape: {adata.shape[0]} cells × {adata.shape[1]} genes")
print(f"\nobs columns: {list(adata.obs.columns)}")
print(f"var columns: {list(adata.var.columns)}")

In [ ]:
# Quick look at the data
adata.obs.head()

In [ ]:
adata.var.head()

In [ ]:
adata.write("tcga_cptac_kidney_data.h5ad", compression="gzip")

In [ ]:
# Read the file back into an AnnData object
adata = ad.read_h5ad("tcga_cptac_kidney_data.h5ad")

# Check the object to make sure it loaded correctly
print(adata)